Install requirements.

In [ ]:
!pip install -r requirements.txt

Download a wandb model below.

In [ ]:
import wandb

# define wandb environment
%env WANDB_LOG_MODEL=true
%env WANDB_API_KEY=53c099408fab02d1e4fff7386e8dfc1e759689a1

In [ ]:
# download wandb model


In [ ]:
wandb.finish()